# Assignment of week 3 of the Project Capstone for Data Science

# PART I

## 0 - Imports the different libraries needed

In [14]:
#!conda install -c conda-forge pandas --yes
#!conda install -c conda-forge numpy --yes
#!conda install -c conda-forge beautifulsoup4 --yes
#!conda install -c conda-forge lxml --yes
#!conda install -c conda-forge geocoder --yes

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests

## 1 - Collects the web page start the scrapping and identify the relevant table

In [15]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'html.parser')

table_html = soup.find('table', class_='wikitable sortable')

## 2 - Creates the raw pandas dataframe df and fill it with the data from the table

In [16]:
columns_name = list(table_html.tbody.find('tr').text.split('\n')[1:4])
df = pd.DataFrame(columns=(columns_name))
i = 0
for row in table_html.tbody.find_all('tr'):
    item = list(row.text.split('\n')[1:4])
    df.loc[i] = item
    i = i+1
df.drop(df.index[0], inplace = True)

df.head()

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


## 3 - Creates a new pandas dataframe, remove the lines without borough and solve the Not assigned Neighbourhood

In [17]:
df_clean = df[df.Borough != 'Not assigned']
df_clean.reset_index(drop= True, inplace = True)

index_NA = df_clean.index[df_clean.Neighbourhood == 'Not assigned'].astype(int)
df_clean.loc[index_NA, 'Neighbourhood'] = df_clean.loc[index_NA, 'Borough']
df_clean.head()

/home/adrien/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


## 4 - Creates a new pandas dataframe df_final and copy paste the different Neighbourhood for the identical postcode

In [18]:
df_proc = pd.DataFrame(columns=(columns_name))

max = df_clean.shape[0]

for index1 in range(max):
    save = df_clean.loc[index1, 'Neighbourhood']
    for index2 in range(index1+1, max):
        if df_clean.loc[index1, 'Postcode'] == df_clean.loc[index2, 'Postcode']:
            save = save +', ' + df_clean.loc[index2, 'Neighbourhood']
        else:
            index2 = index2+1
    
    df_proc.loc[index1, :] = df_clean.loc[index1, :]
    df_proc.loc[index1, 'Neighbourhood'] = save
            
df_proc.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,"Lawrence Heights, Lawrence Manor"


## 5 - Creates a new pandas dataframe df_last with unique value for each Postcode

In [44]:
max = df_proc.shape[0]
data = np.zeros(shape=(max,3))

df_final = pd.DataFrame(data, columns=(columns_name))

pc_final = list(['A', 'B', 'C'])

for index in range(max):
    pc_proc = df_proc.loc[index, 'Postcode']
    if pc_proc not in pc_final:
        df_final.loc[index, :] = df_proc.loc[index, :]
        pc_final = pc_final + df_final.loc[index, 'Postcode'].split()

    else:
        index = index + 1

df_last = df_final[df_final.Borough != 0]
df_last.reset_index(drop=True, inplace = True)

df_last.head()
#df_last.to_csv('test.csv')

## 6 - Displays the shape of the cleaned dataframe

In [20]:
df_last.shape

(103, 3)

# PART II

## 1 - Read the CSV file because geocoder remains stuck

In [29]:
url = 'https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
df_coord = pd.read_csv(url)
df_coord.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
df_coord.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [47]:
df_latlng = df_last.merge(df_coord, on=['Postcode'])
df_latlng.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


# PART III

# 1 - 

In [49]:
toronto_data = df_latlng[df_latlng['Borough'] == 'Central Toronto'].reset_index(drop=True)
toronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307
4,M4R,Central Toronto,North Toronto West,43.715383,-79.405678


In [ ]:
# create map of New York using latitude and longitude values

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_toronto